Het doel van deze notebook is het voorverwerken van de data, zodat deze gereed is voor analyse. Dit doe ik door het raster om te zetten naar een 2D-array van features per pixel, zodat deze kunnen worden geclusterd en geclassificeerd.

In [2]:
# Benodigde imports
import numpy as np
import rasterio
import os
from rasterio.windows import Window
import warnings
from rasterio.errors import NotGeoreferencedWarning
from scipy.ndimage import generic_filter
from tqdm import tqdm
import glob

Met het volgende codeblok maak ik een nieuw TIF-bestand waarbij alle pixels met waarde 0 worden vervangen door `NaN`.  
Deze `NaN`-waarden worden in het rasterbestand ingesteld als NoData-pixels, zodat ze genegeerd worden bij verdere analyse.

In [2]:
warnings.filterwarnings("ignore", category=NotGeoreferencedWarning)

input_path = "../data/private/haarlem_2024_cir.tif"
output_path = "../data/private/haarlem_2024_cir_nan.tif"

with rasterio.open(input_path) as src:
    profile = src.profile.copy()
    profile.update(
        dtype='float32',
        count=3,
        nodata=np.nan,
        compress='deflate',
        predictor=2,
        zlevel=6,
        bigtiff='yes'
    )

    with rasterio.open(output_path, 'w', **profile) as dst:
        # Loop over blokken
        for ji, window in src.block_windows(1):
            data = src.read([1, 2, 3], window=window)

            # Zet 0 om in NaN
            data = np.where(data == 0, np.nan, data).astype('float32')

            # Schrijf blok
            dst.write(data, window=window)

print("Raster opgeslagen als:", output_path)


Raster opgeslagen als: ../data/private/haarlem_2024_cir_nan.tif


In onderstaand codeblok worden twee soorten kenmerken per pixel berekend:

1. **Genormaliseerde RGB-waarden**  
   Elke kleurband (rood, groen, blauw) wordt geschaald naar een bereik tussen 0 en 1.  
   Dit zorgt ervoor dat alle banden evenveel gewicht hebben in de analyse, ongeacht hun oorspronkelijke schaal.

2. **Textuurkenmerken**  
   Voor elke kleurband wordt de lokale standaarddeviatie berekend in een 5×5 venster rondom elke pixel.  
   Deze waarde geeft aan hoeveel variatie (ruwheid of structuur) er in de directe omgeving zit.  
   Pixels in een egaal gebied (zoals asfalt of dakvlak) hebben een lage textuurwaarde, terwijl vegetatie of randen juist hogere waarden hebben.

Beide outputs worden opgeslagen als 3D-arrays met de vorm (3, hoogte, breedte), één voor de genormaliseerde waarden en één voor de textuur.


In [ ]:
def normalize_band(band):
    return (band - np.nanmin(band)) / (np.nanmax(band) - np.nanmin(band))

def local_std(arr, size=5):
    return generic_filter(arr, np.std, size=(size, size), mode='nearest')

input_path = "../data/private/haarlem_2024_cir_nan.tif"
output_dir = "../data/processed/features_parts"
os.makedirs(output_dir, exist_ok=True)

batch_size = 1000
batch = []
batch_idx = 0
pixel_total = 0
blok_teller = 0

with rasterio.open(input_path) as src:
    for i, (ji, window) in enumerate(tqdm(src.block_windows(1), desc="Verwerken van blokken")):
        data = src.read([1, 2, 3], window=window)

        if np.isnan(data).all():
            continue

        mask = ~np.isnan(data).any(axis=0)
        if not np.any(mask):
            continue

        norm = np.stack([normalize_band(b) for b in data])
        texture = np.stack([local_std(b, size=5) for b in data])

        row_idx, col_idx = np.where(mask)
        transform = src.window_transform(window)
        x_coords, y_coords = rasterio.transform.xy(transform, row_idx, col_idx, offset='center')
        x_coords = np.array(x_coords)
        y_coords = np.array(y_coords)

        feats = np.stack([
            norm[0][mask],
            norm[1][mask],
            norm[2][mask],
            texture[0][mask],
            texture[1][mask],
            texture[2][mask],
            x_coords,
            y_coords
        ], axis=1)

        batch.append(feats)
        pixel_total += feats.shape[0]
        blok_teller += 1

        if len(batch) >= batch_size:
            features_block = np.vstack(batch)
            np.save(os.path.join(output_dir, f"features_part_{batch_idx:03}.npy"), features_block)
            print(f"Batch {batch_idx} opgeslagen: {features_block.shape[0]} pixels (totaal: {pixel_total:,} pixels)")
            batch = []
            batch_idx += 1

# Laatste batch wegschrijven
if batch:
    features_block = np.vstack(batch)
    np.save(os.path.join(output_dir, f"features_part_{batch_idx:03}.npy"), features_block)
    print(f"Laatste batch {batch_idx} opgeslagen: {features_block.shape[0]} pixels (totaal: {pixel_total:,} pixels)")

print(f"\n Klaar: {blok_teller} blokken verwerkt, totaal {pixel_total:,} geldige pixels opgeslagen.")

11,5 uur later is bovenstaande verwerking klaar. In de volgende stap voeg ik alle npy-bestanden uit de batches samen tot één bestand.

In [ ]:
input_dir = "../data/processed/features_parts"
output_path = "../data/processed/features_all.npy"

part_files = sorted(glob.glob(os.path.join(input_dir, "features_part_*.npy")))

# Inspecteer eerste bestand om shape te bepalen
first = np.load(part_files[0])
n_features = first.shape[1]
total_rows = sum(np.load(f).shape[0] for f in part_files)

print(f"Totaal: {total_rows:,} rijen × {n_features} features")

# Maak leeg bestand op schijf
output = np.lib.format.open_memmap(
    output_path,
    dtype=first.dtype,
    mode='w+',
    shape=(total_rows, n_features)
)

# Vul het bestand stap voor stap
cursor = 0
for i, file in enumerate(part_files):
    part = np.load(file)
    rows = part.shape[0]
    output[cursor:cursor + rows] = part
    cursor += rows
    print(f"Toegevoegd: {file} → {rows:,} rijen")

# Sluit bestand
del output

print(f"\n Bestand opgeslagen: {output_path}")
